## 一、数据集处理

### 1、将训练集的mentions转换为text对应的标记

In [99]:
import pandas as pd

# 读取训练集
df = pd.read_excel("data/train.xlsx")

# entity的起始和后续标记
B_ENT = 1
I_ENT = 2

# 逐个样本处理
for index, row in df.iterrows():

    # mentions的列表
    mention_list = eval(row['mentions'])

    # text字符串的长度
    text_len = len(row['text'])

    # 创建与text等长的label
    label = [0]*text_len

    # 对mention列表的每一个mention做处理，其中每个mention是一个dict
    for mention in mention_list:

        # mention字典中的'offset'字段表示某个实体的开始和结束位置，左闭右开
        start, end = eval(mention['offset'])

        # entity的起始位置标记为B_ENT
        label[start] = B_ENT
        
        # entity的其余位置标记为I_ENT
        for num in range(start+1, end):
            label[num] = I_ENT

    # 存储label
    df.loc[index, "labels"] = str(label)
    
# 保存为新的CSV文件
df.to_csv("data/train_labeled.csv")


## 二、模型构建

### 1、加载分词器并测试

In [100]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt6')

# 测试文本
text = ["红土创新基金管理有限公司6月30日发布公告，红土创新盐田港REIT(180301)的基金经理新聘陈超。"
,"金能科技: 金能科技股份有限公司关于“金能转债”转股价格调整的提示性公告"]

# 将文本进行逐字分解
def split_sent(text):
    return [char for char in text]

def split(text_list):

    text_split = []
    for sent in text_list:
        sent_split = split_sent(sent)
        text_split.append(sent_split)
    return text_split


# 分解并编码
def split_and_encode(text_list):

    inputs = tokenizer.batch_encode_plus(
    split(text_list),
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

    return inputs

# 展示编码结果
inputs_test = split_and_encode(text)
for key,value in inputs_test.items():
    print(key ,": \n", value, "\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


input_ids : 
 tensor([[ 101, 5273, 1759, 1158, 3173, 1825, 7032, 5052, 4415, 3300, 7361, 1062,
         1385,  127, 3299,  124,  121, 3189, 1355, 2357, 1062, 1440, 8024, 5273,
         1759, 1158, 3173, 4663, 4506, 3949,  160,  147,  151,  162,  113,  122,
          129,  121,  124,  121,  122,  114, 4638, 1825, 7032, 5307, 4415, 3173,
         5470, 7357, 6631,  511,  102],
        [ 101, 7032, 5543, 4906, 2825,  131, 7032, 5543, 4906, 2825, 5500,  819,
         3300, 7361, 1062, 1385, 1068,  754,  100, 7032, 5543, 6760,  965,  100,
         6760, 5500,  817, 3419, 6444, 3146, 4638, 2990, 4850, 2595, 1062, 1440,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]]) 

token_type_ids : 
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### 2、定义新的Dataset类

In [101]:
import torch
import datasets

# Dataset类，用于处理训练集
class Dataset(torch.utils.data.Dataset):

    def __init__(self, path):
        
        # 用pandas读取csv，并将其转换为Dataset
        data_df = pd.read_csv(path)
        data_list = data_df.to_dict(orient="list")
        dataset = datasets.Dataset.from_dict(data_list)

        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        token = split_sent(self.data[index]['text'])
        label = eval(self.data[index]['labels'])

        return token, label 

# 创建数据集实例
dataset = Dataset('data/train_labeled.csv')

# 使用数据集
token, label = dataset[0]

len(dataset), token, label

(500,
 ['大',
  '地',
  '熊',
  '成',
  '功',
  '收',
  '购',
  '技',
  '研',
  '株',
  '式',
  '会',
  '社',
  '安',
  '徽',
  '大',
  '地',
  '熊',
  '新',
  '材',
  '料',
  '股',
  '份',
  '有',
  '限',
  '公',
  '司',
  '2',
  '0',
  '2',
  '3',
  '年',
  '6',
  '月',
  '2',
  '2',
  '日',
  ',',
  '大',
  '地',
  '熊',
  '日',
  '本',
  '株',
  '式',
  '会',
  '社',
  '与',
  '技',
  '研',
  '株',
  '式',
  '会',
  '社',
  '(',
  '英',
  '文',
  '名',
  ':',
  'p',
  '.',
  'm',
  '.',
  'g',
  'i',
  'k',
  'e',
  'n',
  ' ',
  'i',
  'n',
  'c',
  ',',
  '简',
  '称',
  'p',
  'm',
  '公',
  '司',
  ')',
  '原',
  '有',
  '股',
  '东',
  '和',
  '相',
  '关',
  '方',
  '签',
  '订',
  '收',
  '购',
  '合',
  '同',
  '》',
  ',',
  '大',
  '地',
  '熊',
  '日',
  '本',
  '株',
  '式',
  '会',
  '社',
  '全',
  '资',
  '收',
  '购',
  'p',
  'm',
  '公',
  '司',
  '并',
  '即',
  '时',
  '启',
  '动',
  '相',
  '关',
  '手',
  '续',
  '交',
  '接',
  '。',
  ' ',
  'p',
  'm',
  '公',
  '司',
  '是',
  '专',
  '业',
  '从',
  '事',
  '磁',
  '钢',
  '整',
  '体',
  '磁',
  '气',
  '回',


### 3、定义数据整理函数

In [102]:
#数据整理函数
def collate_fn(data):
    
    # tokens是分解后的文本
    tokens = [row[0] for row in data]
    # labels是对应的标记
    labels = [row[1] for row in data]

    # inputs是tokens的编码
    inputs = tokenizer.batch_encode_plus(tokens,
                                         truncation=True,
                                         padding=True,
                                         return_tensors='pt',
                                         is_split_into_words=True)

    # 编码的长度（最长长度）
    length = inputs['input_ids'].shape[1]

    # 将[CLS]和[PAD]标记为3，并将标记长度与编码长度统一
    for i in range(len(labels)):
        labels[i] = [3] + labels[i]
        labels[i] += [3] * length
        labels[i] = labels[i][:length]

    return inputs, torch.LongTensor(labels)


### 4、定义数据加载器

In [103]:
batch_size = 20

#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

#查看数据样例
for i, (inputs, labels) in enumerate(loader):
    break

print(len(loader))
print(tokenizer.decode(inputs['input_ids'][0]))
print(labels[0])

for k, v in inputs.items():
    print(k,'\t', v.shape)

25
[CLS] 影 视 股 集 体 走 高, 万 达 电 影 涨 停, 电 影 市 场 有 望 持 续 回 暖 影 视 股 2 7 日 盘 中 集 体 走 高, 截 至 发 稿, 万 达 电 影 涨 停, 欢 瑞 世 纪 、 唐 德 影 视 涨 超 6 %, 金 逸 影 视 、 幸 福 蓝 海 涨 约 5 %, 中 国 电 影 、 上 海 电 影 、 横 店 影 视 、 博 纳 影 业 、 华 谊 兄 弟 等 涨 超 3 % 。 消 息 面 上, 灯 塔 专 业 版 数 据 显 示, 2 0 2 3 年 端 午 档 期 全 国 票 房 收 入 为 9. 1 亿 元, 相 较 2 0 1 9 年 同 期 增 长 1 5. 9 2 %, 位 列 中 国 影 史 端 午 票 房 第 二 位, 与 2 0 1 8 年 端 午 档 9. 1 2 亿 元 的 最 高 纪 录 基 本 持 平 。 档 期 内 观 影 人 次 达 2 2 5 0. 8 万 人, 相 较 1 9 年 同 期 增 长 0. 8 6 % ; 放 映 场 次 达 1 2 7. 9 万 场, 相 较 1 9 年 同 期 增 长 1 2. 7 9 % ; 上 座 率 为 1 4. 3 %, 已 恢 复 至 接 近 1 9 年 1 5. 5 % 的 水 平 。 今 年 端 午 档 期 观 众 观 影 需 求 强 烈, 电 影 市 场 回 暖 趋 势 明 显 。 影 片 供 给 数 量 方 面, 今 年 端 午 档 期 共 上 映 1 0 部 影 片, 为 历 史 最 高 。 从 头 部 影 片 票 房 收 入 占 比 来 看, 档 期 内 票 房 t o p 2 影 片 为 消 失 的 她 》 ( 5. 0 7 亿 元 ) 、 我 爱 你! 》 ( 1. 0 6 亿 元 ), 两 部 国 产 影 片 票 房 合 计 占 据 总 票 房 近 七 成, 票 房 头 部 效 应 凸 显 。 从 头 部 影 片 内 容 质 量 来 看, 票 房 t o p 3 影 片 淘 票 票 平 均 评 分 为 9. 4 3 分, 同 比 提 升 0. 9 7 分, 相 较 1 9 年 同 期 提 升 1. 3 分, 头 部 影 片 质 量 显 著 提 升, 优 质 内 容 促 进 观 影 需 求 释 放 。 其 中, 国 产 悬

### 5、加载预训练模型

In [105]:
from transformers import AutoModel

# 加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/rbt6')

# 统计参数量
print(sum(i.numel() for i in pretrained.parameters()) / 10000, '万')

#模型试算
pretrained(**inputs_test).last_hidden_state.shape
"""这部分如果把输入改为inputs就不行"""

Some weights of the model checkpoint at hfl/rbt6 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5974.0416 万


'这部分如果把输入改为inputs就不行'